In [342]:
import pandas as pd
import numpy as np

In [320]:
Dataset = pd.read_csv('/content/drive/MyDrive/Kaggle /Titanic/train.csv')
Dataset_Test = pd.read_csv('/content/drive/MyDrive/Kaggle /Titanic/test.csv')

In [ ]:
show_data_visualizations(Dataset)

In [ ]:
Train = Dataset.copy()
Test = Dataset_Test.copy()
Train, Test = basic_preprocessing(Train, Test)
Train, Test = data_imputation_mice(Train, Test)
Y_pred = prediction_model_xgboost( Train, Test, 100 )
create_result_csv(Y_pred)

In [347]:
def basic_preprocessing(dataset, dataset_test):
  dataset = dataset.iloc[:,[1,2,4,5,6,7,9,11]]
  dataset_test = dataset_test.iloc[:,[1,3,4,5,6,8,10]]

  dataset.Sex = pd.Categorical(dataset.Sex)
  dataset['Sex'] = dataset.Sex.cat.codes
  dataset.Embarked = pd.Categorical(dataset.Embarked)
  dataset['Embarked'] = dataset.Embarked.cat.codes

  dataset_test.Sex = pd.Categorical(dataset_test.Sex)
  dataset_test['Sex'] = dataset_test.Sex.cat.codes
  dataset_test.Embarked = pd.Categorical(dataset_test.Embarked)
  dataset_test['Embarked'] = dataset_test.Embarked.cat.codes

  return( dataset, dataset_test )

In [346]:
def moderate_preprocessing():
  dataset = dataset.iloc[:,[1,2,3,4,5,6,7,9,11]]
  dataset_test = dataset_test.iloc[:,[1,2,3,4,5,6,8,10]]

  dataset.Sex = pd.Categorical(dataset.Sex)
  dataset['Sex'] = dataset.Sex.cat.codes
  dataset.Embarked = pd.Categorical(dataset.Embarked)
  dataset['Embarked'] = dataset.Embarked.cat.codes

  dataset_test.Sex = pd.Categorical(dataset_test.Sex)
  dataset_test['Sex'] = dataset_test.Sex.cat.codes
  dataset_test.Embarked = pd.Categorical(dataset_test.Embarked)
  dataset_test['Embarked'] = dataset_test.Embarked.cat.codes

  dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
  dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
    'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
  dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
  dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
  dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
  title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
  dataset['Title'] = dataset['Title'].map(title_mapping)
  dataset = dataset.drop(columns=['Name'])

  dataset_test['Title'] = dataset_test.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
  dataset_test['Title'] = dataset_test['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
    'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
  dataset_test['Title'] = dataset_test['Title'].replace('Mlle', 'Miss')
  dataset_test['Title'] = dataset_test['Title'].replace('Ms', 'Miss')
  dataset_test['Title'] = dataset_test['Title'].replace('Mme', 'Mrs')
  title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
  dataset_test['Title'] = dataset_test['Title'].map(title_mapping)
  dataset_test = dataset_test.drop(columns=['Name'])

  dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
  dataset_test['FamilySize'] = dataset_test['SibSp'] + dataset_test['Parch'] + 1
  
  dataset['IsAlone'] = 0
  dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
  dataset = dataset.drop(columns=['FamilySize'])
  dataset_test['IsAlone'] = 0
  dataset_test.loc[dataset_test['FamilySize'] == 1, 'IsAlone'] = 1
  dataset_test = dataset_test.drop(columns=['FamilySize'])

In [278]:
def preprocess_kaggle( dataset, dataset_test ):

  dataset = dataset.iloc[:,[1,2,3,4,5,6,7,9,11]]
  dataset_test = dataset_test.iloc[:,[1,2,3,4,5,6,8,10]]

  dataset.Sex = pd.Categorical(dataset.Sex)
  dataset['Sex'] = dataset.Sex.cat.codes
  dataset.Embarked = pd.Categorical(dataset.Embarked)
  dataset['Embarked'] = dataset.Embarked.cat.codes

  dataset_test.Sex = pd.Categorical(dataset_test.Sex)
  dataset_test['Sex'] = dataset_test.Sex.cat.codes
  dataset_test.Embarked = pd.Categorical(dataset_test.Embarked)
  dataset_test['Embarked'] = dataset_test.Embarked.cat.codes

  dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
  dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
    'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
  dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
  dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
  dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
  title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
  dataset['Title'] = dataset['Title'].map(title_mapping)
  dataset = dataset.drop(columns=['Name'])

  dataset_test['Title'] = dataset_test.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
  dataset_test['Title'] = dataset_test['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
    'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
  dataset_test['Title'] = dataset_test['Title'].replace('Mlle', 'Miss')
  dataset_test['Title'] = dataset_test['Title'].replace('Ms', 'Miss')
  dataset_test['Title'] = dataset_test['Title'].replace('Mme', 'Mrs')
  title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
  dataset_test['Title'] = dataset_test['Title'].map(title_mapping)
  dataset_test = dataset_test.drop(columns=['Name'])

  dataset['AgeBand'] = pd.cut(dataset['Age'], 5)
  dataset[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)
  dataset = dataset.drop(['AgeBand'], axis=1)

  dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
  dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
  dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
  dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
  dataset.loc[ dataset['Age'] > 64, 'Age']

  dataset_test.loc[ dataset_test['Age'] <= 16, 'Age'] = 0
  dataset_test.loc[(dataset_test['Age'] > 16) & (dataset_test['Age'] <= 32), 'Age'] = 1
  dataset_test.loc[(dataset_test['Age'] > 32) & (dataset_test['Age'] <= 48), 'Age'] = 2
  dataset_test.loc[(dataset_test['Age'] > 48) & (dataset_test['Age'] <= 64), 'Age'] = 3
  dataset_test.loc[ dataset_test['Age'] > 64, 'Age']

  #Adding Family Size and Adding IsAlone
  dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
  dataset_test['FamilySize'] = dataset_test['SibSp'] + dataset_test['Parch'] + 1
  
  dataset['IsAlone'] = 0
  dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
  dataset = dataset.drop(columns=['FamilySize'])
  dataset_test['IsAlone'] = 0
  dataset_test.loc[dataset_test['FamilySize'] == 1, 'IsAlone'] = 1
  dataset_test = dataset_test.drop(columns=['FamilySize'])
  

  dataset = dataset.drop(columns=['SibSp','Parch'])
  dataset_test = dataset_test.drop(columns=['SibSp','Parch'])


  dataset['FareBand'] = pd.qcut(dataset['Fare'], 4)
  dataset[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)
  dataset = dataset.drop(columns=['FareBand'])

  dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0.0
  dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1.0
  dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2.0
  dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3.0
  dataset['Fare'] = dataset['Fare']

  dataset_test.loc[ dataset_test['Fare'] <= 7.91, 'Fare'] = 0.0
  dataset_test.loc[(dataset_test['Fare'] > 7.91) & (dataset_test['Fare'] <= 14.454), 'Fare'] = 1.0
  dataset_test.loc[(dataset_test['Fare'] > 14.454) & (dataset_test['Fare'] <= 31), 'Fare']   = 2.0
  dataset_test.loc[ dataset_test['Fare'] > 31, 'Fare'] = 3.0
  dataset_test['Fare'] = dataset_test['Fare']

  dataset['Age*Class'] = dataset.Age * dataset.Pclass
  dataset_test['Age*Class'] = dataset_test.Age * dataset_test.Pclass

  
  return (dataset, dataset_test)

In [138]:
def data_imputation_mice(dataset, dataset_test):
  !pip install impyute
  from impyute.imputation.cs import mice

  # start the MICE training
  imputed_training=mice(dataset.values)
  imputed_test=mice(dataset_test.values)

  pd.DataFrame(imputed_training).isnull().sum()
  imputed_training = pd.DataFrame(imputed_training)
  imputed_test = pd.DataFrame(imputed_test)

  return ( imputed_training, imputed_test )

In [180]:
def prediction_model_catboost(imputed_training, imputed_test, iterations):
  !pip install catboost
  from catboost import CatBoostClassifier, Pool

  imputed_training_y = imputed_training.iloc[:,0]
  imputed_training =  imputed_training.iloc[:,1:]
  imputed_training_y = imputed_training_y.values
  imputed_training = imputed_training.values

  train_data = Pool(imputed_training,imputed_training_y)
  test_data = Pool(imputed_test)

  model = CatBoostClassifier(iterations=iterations)

  model.fit(train_data)
  Y_pred = model.predict(test_data, prediction_type='Class')

  acc_catboost = round(model.score(imputed_training, imputed_training_y) * 100, 2)
  print( acc_catboost )

  return (Y_pred)

In [181]:
def prediction_model_random_forest(imputed_training, imputed_test, n_estimators):
  from sklearn.ensemble import RandomForestClassifier

  imputed_training_y = imputed_training.iloc[:,0]
  imputed_training =  imputed_training.iloc[:,1:]
  imputed_training_y = imputed_training_y.values
  imputed_training = imputed_training.values

  random_forest = RandomForestClassifier(n_estimators=n_estimators)
  random_forest.fit(imputed_training, imputed_training_y)
  Y_pred = random_forest.predict(imputed_test)

  acc_random_forest = round(random_forest.score(imputed_training, imputed_training_y) * 100, 2)
  print( acc_random_forest )

  return Y_pred

In [319]:
def prediction_model_xgboost(imputed_training, imputed_test, n_estimators):
  #regressor = xgb.XGBClassifier(n_estimators=100, reg_lambda=1, gamma=0, max_depth=3)
  from xgboost import XGBClassifier

  imputed_training_y = imputed_training.iloc[:,0]
  imputed_training =  imputed_training.iloc[:,1:]
  
  imputed_training_y = imputed_training_y.values
  imputed_training = imputed_training.values

  imputed_training = pd.DataFrame(imputed_training)
  imputed_training = imputed_training.values

  imputed_test = imputed_test.values
  imputed_test = pd.DataFrame(imputed_test)
  imputed_test = imputed_test.values

  model = XGBClassifier(n_estimators=n_estimators)
  model.fit(imputed_training, imputed_training_y)
  Y_pred = model.predict(imputed_test)

  acc_xgboost = round(model.score(imputed_training, imputed_training_y) * 100, 2)
  print( acc_xgboost )

  return Y_pred

In [344]:
def show_data_visualizations( train_df ):
  import seaborn as sns
  import matplotlib.pyplot as plt

  g = sns.FacetGrid(train_df, col='Survived')
  g.map(plt.hist, 'Age', bins=20)

  grid = sns.FacetGrid(train_df, col='Survived', row='Pclass', size=2.2, aspect=1.6)
  grid.map(plt.hist, 'Age', alpha=.5, bins=20)
  grid.add_legend();

  grid = sns.FacetGrid(train_df, row='Embarked', col='Survived', size=2.2, aspect=1.6)
  grid.map(sns.barplot, 'Sex', 'Fare', alpha=.5, ci=None)
  grid.add_legend()

  grid = sns.FacetGrid(train_df, row='Embarked', col='Survived', size=2.2, aspect=1.6)
  grid.map(sns.barplot, 'Sex', 'Fare', alpha=.5, ci=None)
  grid.add_legend()

In [144]:
def create_result_csv(Y_pred):
  result = pd.read_csv('/content/drive/MyDrive/Kaggle /Titanic/gender_submission.csv')
  result.Survived = pd.DataFrame(Y_pred.astype('int64'))
  result.to_csv('result.csv',index=False)

In [ ]:
# import sys
# from impyute.imputation.cs import fast_knn
# sys.setrecursionlimit(100000) #Increase the recursion limit of the OS

# # start the KNN training
# imputed_training=fast_knn(train.values, k=30)